In [1]:
import pandas as pd
fptrain = "../../../data/sba_7a_loans_train.parquet"
fptest = "../../../data/sba_7a_loans_test.parquet"
df_train = pd.read_parquet(fptrain)
df_test = pd.read_parquet(fptest)
df = pd.concat([df_train, df_test])
df

,BorrName,BankFDICNumber,BankZip,BorrZip,NaicsCode,FranchiseCode,BusinessAge,LoanStatus,SBAGuaranteedApproval
0,Brothers Freight Management L,Not Applicable,87109,14580,484121.0,Not Applicable,Change of Ownership,PIF,3525000.0
1,EASY SPACE STORAGE LLC,58665,28403,65401,531130.0,Not Applicable,Change of Ownership,PIF,654750.0
2,H&W Endeavors Inc.,6560,43215,77493,449121.0,S0659,"Startup, Loan Funds will Open Business",PIF,150000.0
3,Imagine Technology Group LLC,4767,80202,85226,423420.0,Not Applicable,Existing or more than 2 years old,PIF,3052500.0
4,Zorn Fruherziehung LLC,33555,33880,2301,624410.0,Not Applicable,"Startup, Loan Funds will Open Business",PIF,187500.0
...,...,...,...,...,...,...,...,...,...
4588,IronPlane LLC,4255,4843,4101,454110.0,Not Applicable,Existing or more than 2 years old,PIF,79600.0
4589,ADORE HAIR & NAILS SALON LLC,17308,96813,96826,812112.0,Not Applicable,"Startup, Loan Funds will Open Business",CHGOFF,15000.0
4590,Sunberry Limited Manufacturing,6560,43215,48335,424990.0,Not Applicable,Existing or more than 2 years old,PIF,717750.0
4591,SNFood &amp; Beverage LLC,5304,54220,53023,312140.0,Not Applicable,Unanswered,PIF,175000.0


In [2]:
df["NaicsCode"] = df["NaicsCode"].apply(lambda x: x.replace(".0", ""))
df = df.drop(["BorrName"], axis=1)
dtypes_toset = {"BorrZip": 'str', "BankZip": "str", "BankFDICNumber": 'str',\
                "NaicsCode": 'str', "FranchiseCode": 'str', \
                "BusinessAge" : 'str', "LoanStatus": 'str'}

In [3]:
df = df.astype(dtypes_toset)

In [4]:
df_catvars = pd.DataFrame.from_dict({k: df[k].nunique() for k, v in dtypes_toset.items() if v in ['category', 'str']}, orient="index").reset_index()
df_catvars.columns = ["Attribute", "Unique_Values"]
df_catvars

,Attribute,Unique_Values
0,BorrZip,9057
1,BankZip,1169
2,BankFDICNumber,1116
3,NaicsCode,860
4,FranchiseCode,979
5,BusinessAge,6
6,LoanStatus,2


In [5]:
high_cardinality_attribs = ["BorrZip", "BankZip", "BankFDICNumber", "NaicsCode", "FranchiseCode"]
df.groupby("BorrZip").size()

BorrZip
10001    10
10002     1
10004     1
10005     3
10006     6
         ..
99709     1
99752     1
99801     1
99827     1
99835     1
Length: 9057, dtype: int64

In [6]:
df["BorrZip"] = df["BorrZip"].apply(lambda x : str(x)[:3])
df["BankZip"] = df["BankZip"].apply(lambda x : str(x)[:3])

In [7]:
df_catvars = pd.DataFrame.from_dict({k: df[k].nunique() for k, v in dtypes_toset.items() if v in ['category', 'str']}, orient="index").reset_index()
df_catvars.columns = ["Attribute", "Unique_Values"]
df_catvars

,Attribute,Unique_Values
0,BorrZip,814
1,BankZip,552
2,BankFDICNumber,1116
3,NaicsCode,860
4,FranchiseCode,979
5,BusinessAge,6
6,LoanStatus,2


In [8]:
dfg = df.groupby("BorrZip")["LoanStatus"].value_counts().reset_index()

In [9]:
dfg.columns

Index(['BorrZip', 'LoanStatus', 'count'], dtype='object')

In [10]:
high_cardinality_attribs = ["BorrZip", "BankZip", "BankFDICNumber", "NaicsCode", "FranchiseCode"]
DROP_THRESHOLD = 5
drop_these_records = {}
for attrib in high_cardinality_attribs:
    dfg = df.groupby(attrib, observed=False).size().reset_index()
    dfg.columns = [attrib, "group_size"]
    drop_these_records[attrib] = dfg[dfg.group_size < DROP_THRESHOLD][attrib].tolist()
    

In [11]:
for attrib in high_cardinality_attribs:
    df = df[~df[attrib].isin(drop_these_records[attrib])]


In [12]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, test_size=0.2)

In [13]:
cols_to_encode = high_cardinality_attribs + ["BusinessAge"]
import category_encoders as ce
encoder = ce.WOEEncoder(cols=cols_to_encode )

In [14]:
df_train["LoanStatus"] = df_train["LoanStatus"].apply(lambda x: 0 if x == "PIF" else 1)
df_test["LoanStatus"] = df_test["LoanStatus"].apply(lambda x: 0 if x == "PIF" else 1)

In [15]:
df_train["LoanStatus"].value_counts() 

LoanStatus
0    14896
1      716
Name: count, dtype: int64

In [16]:
df_train = encoder.fit_transform(df_train, df_train["LoanStatus"])
df_test = encoder.transform(df_test)

In [17]:
fptrain = "../../../data/cleaned_sba_7a_loans_train.parquet"
fptest = "../../../data/cleaned_sba_7a_loans_test.parquet"
df_train.to_parquet(fptrain, index=False)
df_test.to_parquet(fptest, index=False)

In [18]:
df_train

,BankFDICNumber,BankZip,BorrZip,NaicsCode,FranchiseCode,BusinessAge,LoanStatus,SBAGuaranteedApproval
6816,-0.009500,0.027150,-0.591828,0.320766,-0.015009,0.409061,0,6675.0
2514,0.407844,0.429823,0.317818,-0.637439,-0.015009,-0.250978,0,127500.0
16056,-0.212449,-0.817635,0.506784,-0.368685,-0.015009,-0.250978,0,2065500.0
11702,0.324462,-0.145541,1.474368,-1.014041,-0.015009,-0.578405,0,1130925.0
1929,-0.009500,0.027150,-0.351878,-0.701181,-0.015009,-0.578405,0,37500.0
...,...,...,...,...,...,...,...,...
4294,-1.771508,-1.944221,0.393455,0.093939,-0.015009,-0.250978,0,375000.0
1197,0.015906,-0.010848,0.199299,1.110041,-0.015009,-0.250978,0,72000.0
2039,0.835288,-1.186995,-1.126370,0.393455,1.086603,0.315757,0,152025.0
10478,1.969793,1.689599,-0.263324,-0.225584,-0.015009,0.567647,0,2500.0
